<a href="https://colab.research.google.com/github/thor4/neuralnets/blob/master/projects/1-CNN/step5-test_iterations-cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Iterations
--- 
Here we will process the 9 tilt/contrast combinations for all iterations trained in step 4 and save the results to excel for visualization in step 6.
## Import libraries

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 28 14:21:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.special import expit #import sigmoid func
from google.colab import files
tf.random.set_seed(42) #set random seed for reproducibility

### 1. Load and prepare the data
---

### Download the models

These models were created using the `step4_train_iterations` Jupyter notebook. Run the cell to download a zip file from OSF then extract its contents into the newly created directory.

cifar10 gabors models: `content/cifar10_gabors-model_#/`

where # = [0,29]

In [ ]:
# @title Download + unzip models

import requests, os
from zipfile import ZipFile

print("Start downloading and unzipping vanilla models trained on Gabors...")
name = 'cifar10_gabors-models'
fname = f"{name}.zip"
url = f"https://osf.io/e95bx/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall() #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")
!unzip 'cifar10_gabors-model_*.zip'

Start downloading and unzipping vanilla models trained on Gabors...
Download completed.
Archive:  cifar10_gabors-model_1.zip
   creating: cifar10_gabors-model_1/
  inflating: cifar10_gabors-model_1/saved_model.pb  
   creating: cifar10_gabors-model_1/assets/
   creating: cifar10_gabors-model_1/variables/
  inflating: cifar10_gabors-model_1/variables/variables.index  
  inflating: cifar10_gabors-model_1/variables/variables.data-00000-of-00001  
  inflating: cifar10_gabors-model_1/keras_metadata.pb  

Archive:  cifar10_gabors-model_17.zip
   creating: cifar10_gabors-model_17/
   creating: cifar10_gabors-model_17/assets/
  inflating: cifar10_gabors-model_17/saved_model.pb  
   creating: cifar10_gabors-model_17/variables/
  inflating: cifar10_gabors-model_17/variables/variables.index  
  inflating: cifar10_gabors-model_17/variables/variables.data-00000-of-00001  
  inflating: cifar10_gabors-model_17/keras_metadata.pb  

Archive:  cifar10_gabors-model_0.zip
   creating: cifar10_gabors-model

In [ ]:
# @title Delete all zip files to save space.
dir_name = os.getcwd()
test = os.listdir(dir_name)
for item in test:
    if item.endswith(".zip"):
        os.remove(os.path.join(dir_name, item))

In [ ]:
# @title Load the models
model0 = tf.keras.models.load_model('cifar10_gabors-model_0')
model1 = tf.keras.models.load_model('cifar10_gabors-model_1')
model2 = tf.keras.models.load_model('cifar10_gabors-model_2')
model3 = tf.keras.models.load_model('cifar10_gabors-model_31') #replaced chance level originals
model4 = tf.keras.models.load_model('cifar10_gabors-model_4')
model5 = tf.keras.models.load_model('cifar10_gabors-model_5')
model6 = tf.keras.models.load_model('cifar10_gabors-model_6')
model7 = tf.keras.models.load_model('cifar10_gabors-model_7')
model8 = tf.keras.models.load_model('cifar10_gabors-model_8')
model9 = tf.keras.models.load_model('cifar10_gabors-model_9')
model10 = tf.keras.models.load_model('cifar10_gabors-model_10')
model11 = tf.keras.models.load_model('cifar10_gabors-model_32') #replaced chance level originals
model12 = tf.keras.models.load_model('cifar10_gabors-model_12')
model13 = tf.keras.models.load_model('cifar10_gabors-model_13')
model14 = tf.keras.models.load_model('cifar10_gabors-model_14')
model15 = tf.keras.models.load_model('cifar10_gabors-model_15')
model16 = tf.keras.models.load_model('cifar10_gabors-model_16')
model17 = tf.keras.models.load_model('cifar10_gabors-model_17')
model18 = tf.keras.models.load_model('cifar10_gabors-model_18')
model19 = tf.keras.models.load_model('cifar10_gabors-model_19')
model20 = tf.keras.models.load_model('cifar10_gabors-model_20')
model21 = tf.keras.models.load_model('cifar10_gabors-model_21')
model22 = tf.keras.models.load_model('cifar10_gabors-model_22')
model23 = tf.keras.models.load_model('cifar10_gabors-model_23')
model24 = tf.keras.models.load_model('cifar10_gabors-model_24')
model25 = tf.keras.models.load_model('cifar10_gabors-model_25')
model26 = tf.keras.models.load_model('cifar10_gabors-model_26')
model27 = tf.keras.models.load_model('cifar10_gabors-model_27')
model28 = tf.keras.models.load_model('cifar10_gabors-model_28')
model29 = tf.keras.models.load_model('cifar10_gabors-model_29')
model0.summary() #verify architecture

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 80)                2193440   
                                                                 
 dense_4 (Dense)             (None, 1)                 81        
                                                                 
Total params: 2,193,521
Trainable params: 81
Non-trainable params: 2,193,440
_________________________________________________________________


Download the 9 test datasets from OSF and extract the contents into the newly created directory: `content/datasets/`

In [ ]:
# @title Download datasets to test the model

print("Start downloading and unzipping `9 datasets`...")
name = 'tilt_contrast-cifar10'
fname = f"{name}.zip"
url = f"https://osf.io/agxyp/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall("datasets") #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping `9 datasets`...
Download completed.


Load all 9 training sets and use prefetch to streamline image loading.

In [ ]:
# @title Load datasets into tensorflow

from tensorflow.keras.preprocessing import image_dataset_from_directory

BATCH_SIZE = 32 
IMG_SIZE = (160, 160) #forces a resize from 170x170 since MobileNetV2 has weights only for certain sizes
AUTOTUNE = tf.data.AUTOTUNE #prompts the tf.data runtime to tune the value dynamically at runtime
def model2_init_sets(BATCH_SIZE, IMG_SIZE, AUTOTUNE):
    curr_dir = os.getcwd() 
    set1_dir = os.path.join(curr_dir, 'datasets/t_1_0833-c_0_3')
    set2_dir = os.path.join(curr_dir, 'datasets/t_1_0833-c_0_45')
    set3_dir = os.path.join(curr_dir, 'datasets/t_1_0833-c_1')
    set4_dir = os.path.join(curr_dir, 'datasets/t_2_3958-c_0_3')
    set5_dir = os.path.join(curr_dir, 'datasets/t_2_3958-c_0_45')
    set6_dir = os.path.join(curr_dir, 'datasets/t_2_3958-c_1')
    set7_dir = os.path.join(curr_dir, 'datasets/t_4-c_0_3')
    set8_dir = os.path.join(curr_dir, 'datasets/t_4-c_0_45')
    set9_dir = os.path.join(curr_dir, 'datasets/t_4-c_1')
    set1 = image_dataset_from_directory(set1_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE) #2000 images 2 classes
    set2 = image_dataset_from_directory(set2_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)    
    set3 = image_dataset_from_directory(set3_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set4 = image_dataset_from_directory(set4_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set5 = image_dataset_from_directory(set5_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set6 = image_dataset_from_directory(set6_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set7 = image_dataset_from_directory(set7_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE) 
    set8 = image_dataset_from_directory(set8_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set9 = image_dataset_from_directory(set9_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    class_names = set1.class_names #extract class names loading function inferred from subdir's
    set1 = set1.prefetch(buffer_size=AUTOTUNE) 
    set2 = set2.prefetch(buffer_size=AUTOTUNE) 
    set3 = set3.prefetch(buffer_size=AUTOTUNE) 
    set4 = set4.prefetch(buffer_size=AUTOTUNE) 
    set5 = set5.prefetch(buffer_size=AUTOTUNE) 
    set6 = set6.prefetch(buffer_size=AUTOTUNE) 
    set7 = set7.prefetch(buffer_size=AUTOTUNE) 
    set8 = set8.prefetch(buffer_size=AUTOTUNE) 
    set9 = set9.prefetch(buffer_size=AUTOTUNE) 
    return set1,set2,set3,set4,set5,set6,set7,set8,set9,class_names

set1,set2,set3,set4,set5,set6,set7,set8,set9,class_names = model2_init_sets(BATCH_SIZE, IMG_SIZE, AUTOTUNE)

Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.


There are 4,000 images per set, 2,000 per class, 36,000 total across sets.

### 2. Generate logits
Next, we can define a function for processing a dataset through a model.

In [ ]:
def process_dataset(dataset, model):
    all_logits=tf.zeros([], tf.float64) #initialize array to hold all prediction logits (single element)
    all_labels=tf.zeros([], tf.float64) #initialize array to hold all actual labels (single element)
    for image_batch, label_batch in dataset.as_numpy_iterator():
        predictions = model.predict_on_batch(image_batch).flatten() #run batch through model and return logits
        all_logits = tf.experimental.numpy.append(all_logits, predictions)
        all_labels = tf.experimental.numpy.append(all_labels, label_batch)
    #tf.size(all_pred) #1335 elements, 1334 images + 1 placeholder 0 at beginning
    all_logits = all_logits[1:]
    all_labels = all_labels[1:]
    all_logits_sig = expit(all_logits) #sigmoid-transform the logits
    all_pred = np.where((all_logits_sig < 0.5), 0, 1) #replace predictions with 0 or 1
    all_acc = np.where((all_pred == all_labels), 1, 0) #decide whether pred = label
    return all_logits,all_labels,all_pred,all_acc

Next, let's define a function for processing all 9 datasets through a model. In it, we will stack each sets' 4,000 logits in a single dataframe, `df_set`. The resulting dataframe will have 36,000 logits resulting from vertically stacking the logits for set1, set 2, .., set9. Additionally, we will calculate the average raw confidence and accuracy scores for each tilt/contrast combination and save this in a separate dataframe called `df_model_results`.

In [ ]:
def process_model(model,set1,set2,set3,set4,set5,set6,set7,set8,set9):
    df_set = pd.DataFrame(columns=['Logits','Labels','Predictions','Accuracy','Tilt',
                              'Contrast']) #init dataframe
    df_model_results = pd.DataFrame(columns=['Accuracy','Confidence']) #init dataframe
    all_sets = [set1,set2,set3,set4,set5,set6,set7,set8,set9]
    all_tilts = [1.0833, 1.0833, 1.0833, 2.3958, 2.3958, 2.3958, 4, 4, 4]
    all_contrasts = [0.3, 0.45, 1, 0.3, 0.45, 1, 0.3, 0.45, 1]
    for idx, dataset in enumerate(all_sets): #run for all sets:
        tilt = all_tilts[idx]
        contrast = all_contrasts[idx]
        all_logits,all_labels,all_pred,all_acc = process_dataset(dataset, model)
        tilts = np.repeat(tilt, all_pred.size)
        contrasts = np.repeat(contrast, all_pred.size)
        df_set = pd.concat([df_set, pd.DataFrame({'Logits':all_logits.numpy(),
                                      'Labels':all_labels.numpy(),
                                      'Predictions':all_pred,'Accuracy':all_acc,
                                      'Tilt':tilts,'Contrast':contrasts})], 
                  axis=0, ignore_index=True) #append logits, labels,etc to dataframe
        acc = all_acc.mean() #calculate avg accuracy
        conf = np.absolute(all_logits.numpy()).mean()
        df_model_results = pd.concat([df_model_results, pd.DataFrame({'Accuracy':[acc],
                                                'Confidence':[conf],
                                                'Tilt':tilt,'Contrast':contrast})], 
                      axis=0, ignore_index=True) #append acc & conf to dataframe
    return df_set,df_model_results

Finally, we will process all model iterations and export the resulting logits and tilt x accuracy results as excel spreadsheets.

In [ ]:
all_models=[model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,
            model10,model11,model12,model13,model14,model15,model16,model17,model18,model19,
            model20,model21,model22,model23,model24,model25,model26,model27,model28,model29]
for idx, model in enumerate(all_models): #run for all models:
  df_set,df_model_results = process_model(model,set1,set2,set3,set4,set5,set6,set7,set8,set9)
  df_results_groupbyta = df_model_results.reset_index().set_index(['Tilt','Contrast'])
  del df_results_groupbyta['index']
  model_logits = "cifar10_gabors-model_{}-logits.xlsx".format(idx)
  logits_excel_filepath = os.path.join(os.getcwd(), model_logits) #prep path to save to
  df_set.to_excel(logits_excel_filepath, index=False) #save to disk
  model_results = "cifar10_gabors-model_{}-results.xlsx".format(idx)
  results_excel_filepath = os.path.join(os.getcwd(), model_results) #prep path to save to
  df_results_groupbyta.to_excel(results_excel_filepath, index=True) #save to disk

In [ ]:
!zip -r "cifar10_gabors-models-results_and_logits.zip"  . -i "*.xlsx"

  adding: cifar10_gabors-model_11-results.xlsx (deflated 10%)
  adding: cifar10_gabors-model_19-logits.xlsx (deflated 19%)
  adding: cifar10_gabors-model_23-results.xlsx (deflated 10%)
  adding: cifar10_gabors-model_9-logits.xlsx (deflated 20%)
  adding: cifar10_gabors-model_10-logits.xlsx (deflated 20%)
  adding: cifar10_gabors-model_8-logits.xlsx (deflated 20%)
  adding: cifar10_gabors-model_17-logits.xlsx (deflated 20%)
  adding: cifar10_gabors-model_1-logits.xlsx (deflated 19%)
  adding: cifar10_gabors-model_0-results.xlsx (deflated 11%)
  adding: cifar10_gabors-model_24-logits.xlsx (deflated 19%)
  adding: cifar10_gabors-model_29-results.xlsx (deflated 10%)
  adding: cifar10_gabors-model_13-logits.xlsx (deflated 19%)
  adding: cifar10_gabors-model_21-results.xlsx (deflated 10%)
  adding: cifar10_gabors-model_4-results.xlsx (deflated 10%)
  adding: cifar10_gabors-model_3-logits.xlsx (deflated 20%)
  adding: cifar10_gabors-model_19-results.xlsx (deflated 10%)
  adding: cifar10_gabor

In [ ]:
files.download("cifar10_gabors-models-results_and_logits.zip") #save all the results

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>